In [ ]:
import numpy as np
import math
from scipy.special import gammaln, gammaincc, gammainc

def Pr(u, eta):
    """
    Compute probability Pr(u, eta) as defined in the original C function.
    """
    if u == 0:
        return math.exp(-eta)
    else:
        sum_p = 0.0
        for l in range(1, u+1 ):
            sum_p += math.exp(-eta - u * math.log(2) + l * math.log(eta)
                              - gammaln(l + 1) + gammaln(u) - gammaln(l) - gammaln(u - l + 1))
        return sum_p

def OverlappingTemplateMatchings(n, epsilon):
    """
    Overlapping Template Matching Test (Python equivalent of the C function).

    Parameters:
    m        : Length of the template (block of 1s)
    n        : Length of the sequence
    epsilon  : Binary sequence to test
    """
    M = 1032
    N = n // M
    K = 5
    nu = np.zeros(6, dtype=int)
    pi = np.array([0.364091, 0.185659, 0.139381, 0.100571, 0.0704323, 0.139865])

    sequence = np.ones(m, dtype=int)

    lambda_val = (M - m + 1) / (2 ** m)
    eta = lambda_val / 2.0

    # Compute Probabilities
    sum_pi = 0.0
    for i in range(K):
        pi[i] = Pr(i, eta)
        sum_pi += pi[i]
    pi[K] = 1 - sum_pi

    for i in range(N):
        W_obs = 0
        for j in range(M - m + 1):
            if np.array_equal(sequence, epsilon[i * M + j: i * M + j + m]):

                W_obs += 1

        if W_obs <= 4:
            nu[W_obs] += 1
        else:
            nu[K] += 1
 
    # Compute Chi-square statistic
    chi2 = sum(((nu[i] - N * pi[i]) ** 2) / (N * pi[i]) for i in range(len(nu)))
    p_value = gammaincc(K / 2.0, chi2 / 2.0)

    # Print results
    print("\n\tOVERLAPPING TEMPLATE MATCHING TEST")
    print("-------------------------------------------------")
    print(f"\t(a) n (sequence length)      = {n}")
    print(f"\t(b) m (block length of 1s)   = {m}")
    print(f"\t(c) M (length of substring)  = {M}")
    print(f"\t(d) N (number of substrings) = {N}")
    print(f"\t(e) lambda [(M-m+1)/2^m]     = {lambda_val}")
    print(f"\t(f) eta                      = {eta}")
    print("-------------------------------------------------")
    print("\t   FREQUENCY")
    print("\t  0   1   2   3   4  >=5   Chi^2   P-value  Assignment")
    print("-------------------------------------------------")
    print(f"\t{nu[0]:3d} {nu[1]:3d} {nu[2]:3d} {nu[3]:3d} {nu[4]:3d} {nu[5]:3d}  {chi2:.6f}  {p_value:.6f}  {'FAILURE' if p_value < 0.01 else 'SUCCESS'}")

    return p_value
